In [1]:
import re
import pickle
from heapq import nlargest
import time

import pandas as pd
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import session_info

In [2]:
# Посмотрим версии используемых модулей.
session_info.show()

In [3]:
# Загрузим ранее сохранённые уникальные навазния компаний.
text_unic = pickle.load(open('companies_name_unic.pickle','rb'))

# Загрузим ранее сохранённый tfidfvectorizer.
my_vectorizer = pickle.load(open('vectorizer_2.pickle','rb'))

def sentence_embedding_fl(string, vectorizer):
    tf_vectorizer = vectorizer
    sentence_list = [tf_vectorizer.transform([word]).toarray() for word in string.split()]
        
    result = np.sum(np.array(sentence_list), axis=0)
    if len(result.shape) == 0:
        result = np.zeros((1,9776))
    
    return result

In [4]:
# Загрузим ранее сохранённые эмбеддинги для названий компаний.
name_embeddings = pickle.load(open('name_embeddings.pickle','rb'))

In [5]:
# Топ для вывода и порог для косинусного сходства.
top = 5
treshold_for_cosine_similarity = 0.5

# Функция для поиска топ 5 схожих названий и их вывода.
def find_top5(text, top, treshold):
    
    # Получим эмбеддинг текста
    text_vector = sentence_embedding_fl(text, my_vectorizer)
    
    # Вычислим косинусное сходство 
    cosine_similarity_list = [cosine_similarity( text_vector, sentence_vector)[0] for sentence_vector in name_embeddings ]
    
    # Ищем схожие названия
    company_names = []
    maximums = nlargest(top, cosine_similarity_list)
    maximums_treshold = [max for max in maximums if max >= treshold ]
    if len(maximums_treshold) > 0:
            indexes_set = set()
            for maximum in maximums_treshold:
                indexes = list(np.where(np.array(cosine_similarity_list) == maximum)[0])
                for index in indexes:
                    if index not in indexes_set:
                        indexes_set.add(index)
                        company_names.append(text_unic[index])

    if len(company_names) < top:
        result = company_names
    else:
        result = company_names[:top]
        
    
    print("Косинусное сходство ", [round(float(element),3) for element in maximums])
    print()

    if len(result) > 0:
        for name in result:
            print(name)
    else:
        print('Похожих названий, превышающих порог ',treshold, " не обнаружено." )
        
        
        
# Второй вариант функции, добавленный для работы с функцией main()        
def find_top(text, top, treshold):
    
    # Получим эмбеддинг текста
    text_vector = sentence_embedding_fl(text, my_vectorizer)
    
    # Вычислим косинусное сходство 
    cosine_similarity_list = [cosine_similarity( text_vector, sentence_vector)[0] for sentence_vector in name_embeddings ]
    
    # Ищем схожие названия
    company_names = []
    maximums = nlargest(top, cosine_similarity_list)
    maximums_treshold = [max for max in maximums if max >= treshold ]
    if len(maximums_treshold) > 0:
            indexes_set = set()
            for maximum in maximums_treshold:
                indexes = list(np.where(np.array(cosine_similarity_list) == maximum)[0])
                for index in indexes:
                    if index not in indexes_set:
                        indexes_set.add(index)
                        company_names.append(text_unic[index])

    if len(company_names) < top:
        result = company_names
    else:
        result = company_names[:top]
    
    return result

In [6]:
# Время начала обработки поиска. 
start_time = time.time()

# Тестовый текст 1. Сначала возьмём порог 0.5 для косинусного сходства.
my_test_text_1 = 'Darth Vader is a great Sith Lord'

# Найдем схожие названия
find_top5(my_test_text_1, top, treshold_for_cosine_similarity)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.447, 0.447, 0.447, 0.355, 0.316]

Похожих названий, превышающих порог  0.5  не обнаружено.

Время обработки поиска =  12.123


In [7]:
# Видим, что ничего не найдено, так косинусное сходство меньше порога.

In [8]:
# Время начала обработки поиска. 
start_time = time.time()

treshold_for_cosine_similarity = 0.3

# Тестовый текст 1. Теперь будем использовать порог 0.3.
my_test_text_1 = 'Darth Vader is a great Sith Lord'

find_top5(my_test_text_1, top, treshold_for_cosine_similarity)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.447, 0.447, 0.447, 0.355, 0.316]

Lord Industrial Ltda
Lord India Pvt., Ltd.
Jm Lord International Llc
Greatech Philippines Inc.
Great Sports Infra Pvt., Ltd.

Время обработки поиска =  11.422


In [9]:
# Видим, что к непохожему на название компании тексту нашлись названия, где фигурируют похожие слова "Lord" и "Great".

In [10]:
# Время начала обработки поиска. 
start_time = time.time()

# Теперь попробуем текст, который был в датасете.
my_test_text_2 = 'Powermax Rubber Factory'

find_top5(my_test_text_2, top, treshold_for_cosine_similarity)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.816, 0.577, 0.577, 0.577, 0.577]

Powermax Rubber Factory
B G N Rubber Factory
Western Rubbers India Pvt., Ltd.
K.S. Rubbers
Imp. Rubbers International Llc

Время обработки поиска =  10.677


In [11]:
# Видим что на первом месте исходный текст (не удивительно), а дальше тексты с похожими словами. 

In [12]:
# Время начала обработки поиска. 
start_time = time.time()

# Теперь попробуем текст, не был в датасете, но похож, на то, что был там.
# Оригинал: Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
# Изменённый вариант
my_test_text_3 = 'Well Aero System Lab Industrial Craft'

find_top5(my_test_text_3, top, treshold_for_cosine_similarity)

# Время окончания обработки поиска.
end_time = time.time()

print()
print("Время обработки поиска = ", round(end_time-start_time, 3))

Косинусное сходство  [0.494, 0.45, 0.408, 0.408, 0.408]

Honeywell Aerospace Systems Laboratory Co. S. De .R.L. De C.V.
Aerocosta Global Systems Inc.
Craft Colombia Sas
Mx Systems International P Ltd.
Craft Argentina S.A.

Время обработки поиска =  10.659


In [13]:
# Видим что на первом месте текст, который был изменён а дальше тексты с похожими словами. 

In [14]:
# Время поиска и вывода похожих названий около 11 секунд. Долго потому что каждый раз поиск запускается для 18022 эмбеддингов.

In [15]:
# Функция позволит вводить строку для поиска с клавиатуры.
def main():
    while True:
        comp_name = input("Введите название компании:\n\n").rstrip()
        if comp_name == "exit":
            break
        k = 10
        
        start_time = time.time()
        
        try:
            top_comp = find_top(comp_name, k, treshold_for_cosine_similarity) 
        except ValueError:
            print ("Похожих компаний нет в списке или введены кракозябры\n\n")
            continue
        print (f"Топ {k} похожих компаний:\n\n")
        for i, comp in enumerate(top_comp):
            print (f"{i + 1}: {comp}")
        print("\n")
        
        end_time = time.time()

        print("Время обработки поиска = ", round(end_time-start_time, 3))
        print()

In [16]:
# После ввода текста приходится ждать около 10 секунд.
main()

Введите название компании:

Golden Steel Industrie
Топ 10 похожих компаний:


1: Golden Trading Ltd.
2: Golden International Group
3: Qingdao Kingdom Plastic&Steel Co. L
4: Emirates Steel Industries
5: Qatar Steel Co.
6: PR INDUSTRIE
7: Golden Wheel Industrial Mfg. Corp
8: M Industria S.A De C.V.
9: P.R.INDUSTRIE Produits Routiers Industrie
10: Industrias Japan S.A.


Время обработки поиска =  10.676

Введите название компании:

Black Water Company
Топ 10 похожих компаний:


1: Black Rubber India
2: International Rubber Company
3: International Rubber Company LLC
4: Zhengzhou Water Supply Engineering Company
5: Ltd. Liability Companywater
6: New Materials Co., Ltd. Huizhou City Water
7: The Garland Company, Inc.
8: Sewing Co., Ltd. Qingdao Water
9: The Walking Company
10: Double E Company LLC


Время обработки поиска =  10.73

Введите название компании:

exit
